# Layered Queueing Network with Cache - Single Host

This example demonstrates a layered queueing network (LQN) with a cache task on a single host. The model includes cache hit/miss activities with different service times.

In [ ]:
// Kotlin notebook
import jline.*
import jline.lang.*
import jline.lang.nodes.*
import jline.lang.processes.*
import jline.lang.constant.*
import jline.lang.layered.*
import jline.solvers.ln.*
import jline.solvers.mva.*
import jline.util.matrix.*

In [ ]:
val model = LayeredNetwork("cacheInLayeredNetwork")

// Client processor and task
val P1 = Processor(model, "P1", 1, SchedStrategy.PS)
val T1 = Task(model, "T1", 1, SchedStrategy.REF).on(P1)
val E1 = Entry(model, "E1").on(T1)

println("Client Configuration:")
println("- Processor P1: 1 server, PS scheduling")
println("- Task T1: 1 user, REF scheduling")
println("- Entry E1: Client entry point")

In [ ]:
// Cache task configuration
val totalItems = 4
val cacheCapacity = 2

// Create uniform access probabilities
val accessProbs = Matrix(1, totalItems).fill(1.0 / totalItems)
val pAccess = DiscreteSampler(accessProbs)

val PC = Processor(model, "PC", 1, SchedStrategy.PS)
val C2 = CacheTask(model, "C2", totalItems, cacheCapacity, ReplacementStrategy.RR, 1).on(PC)
val I2 = ItemEntry(model, "I2", totalItems, pAccess).on(C2)

println("\nCache Configuration:")
println("- Total items: $totalItems")
println("- Cache capacity: $cacheCapacity")
println("- Replacement strategy: Round Robin (RR)")
println("- Access pattern: Uniform (${1.0/totalItems} probability per item)")
println("- Processor PC: 1 server, PS scheduling")
println("- Cache tokens: 1")

In [ ]:
// Define activities
val A1 = Activity(model, "A1", Immediate()).on(T1).boundTo(E1).synchCall(I2, 1)
val AC2 = Activity(model, "AC2", Immediate()).on(C2).boundTo(I2)
val AC2h = Activity(model, "AC2h", Exp(1.0)).on(C2).repliesTo(I2)  // Cache hit
val AC2m = Activity(model, "AC2m", Exp(0.5)).on(C2).repliesTo(I2)  // Cache miss

// Add cache access precedence
C2.addPrecedence(ActivityPrecedence.CacheAccess(AC2, arrayOf(AC2h, AC2m)))

println("\nActivity Configuration:")
println("- A1: Client activity (immediate) calls cache")
println("- AC2: Cache access activity (immediate)")
println("- AC2h: Cache hit activity (Exp(1.0) service)")
println("- AC2m: Cache miss activity (Exp(0.5) service)")
println("- Cache access precedence: AC2 → {AC2h, AC2m}")

In [ ]:
// Solve with Layered Network solver using MVA
try {
    val lnOptions = LN.defaultOptions()
    lnOptions.verbose = true
    
    val mvaOptions = MVA.defaultOptions()
    mvaOptions.verbose = false
    
    val solver = LN(model) { subModel -> MVA(subModel, mvaOptions) }
    solver.options = lnOptions
    
    val avgTable = solver.avgTable
    println("\n=== Layered Network Results ===")
    avgTable.print()
    
    // Analyze cache performance
    println("\nCache Performance Analysis:")
    println("Expected hit ratio with uniform access and RR replacement:")
    val expectedHitRatio = cacheCapacity.toDouble() / totalItems
    println("- Theoretical hit ratio: $expectedHitRatio")
    println("- Cache hit service rate: 1.0 (faster)")
    println("- Cache miss service rate: 0.5 (slower)")
    println("- Miss penalty: 2x slower than hit")
    
} catch (e: Exception) {
    println("\nLN failed: ${e.message}")
    println("This may be due to layered network solver limitations.")
}

In [ ]:
// Display model structure analysis
println("\n=== Layered Network Structure Analysis ===")
println("This model demonstrates:")
println("\n1. Single Host Architecture:")
println("   - Client and cache reside on the same processor")
println("   - No network delays or remote calls")
println("   - Local cache access patterns")
println("\n2. Cache Task Modeling:")
println("   - CacheTask with $totalItems items and capacity $cacheCapacity")
println("   - Round Robin replacement policy")
println("   - Uniform access pattern (zipf parameter = 0)")
println("   - Cache hit ratio ≈ $cacheCapacity/$totalItems = ${cacheCapacity.toDouble()/totalItems}")
println("\n3. Performance Characteristics:")
println("   - Hit latency: Exp(1.0) - relatively fast")
println("   - Miss latency: Exp(0.5) - 2x slower than hit")
println("   - No backend service for misses (simplified model)")
println("\n4. Layered Network Features:")
println("   - Synchronous calls between layers")
println("   - Activity precedences for cache logic")
println("   - Entry-to-entry communication")
println("\nApplications:")
println("- Web server with local cache")
println("- Database with buffer pool")
println("- CPU with L1/L2 cache")
println("- Application with memory cache")

This example demonstrates:

1. **Layered Queueing Network (LQN)**: 
   - Hierarchical model with client and cache layers
   - Entry-to-entry synchronous communication
   - Activity precedences to model cache logic
2. **Cache Task Modeling**:
   - CacheTask with configurable capacity and replacement strategy
   - ItemEntry for uniform access pattern
   - Separate activities for cache hits and misses
3. **Single Host Configuration**:
   - Client and cache on same processor (no network delays)
   - Local cache access patterns
   - Simplified model without backend services
4. **Performance Analysis**:
   - Cache hit ratio depends on capacity vs. total items
   - Different service times for hits vs. misses
   - Impact of cache performance on overall response time

This model is useful for:
- Analyzing local cache effectiveness
- Understanding cache sizing decisions
- Evaluating replacement strategies
- Studying hit/miss ratio impact on performance
- Designing cache-aware applications